In [1]:
#%reset -f
%matplotlib inline
import math as mt
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from numba import jit

<font size = 5> Null Flux Boundary Conditions </font>

In [ ]:
class parameters:
  def __init__(self,c,nu,h,a,b,d1,d2,size,T,L):
    self.c = c
    self.nu = nu
    self.h = h
    self.a = a
    self.b = b
    self.d1 = d1
    self.d2 = d2
    self.size = size
    self.T = T
    self.L = L

  def values(self):
    dx = self.L / self.size  # Spatial Step
    dt = 1*10**(-3)  # Time Step
    n = int(self.T / dt)  # Total number of iterations
    return self.c,self.nu,self.h,self.a,self.b,self.d1,self.d2,self.size,self.T,self.L,dx,dt,n

  def initial_conditions(self):
    U = np.random.rand(self.size_partition, self.size_partition)
    V = np.random.rand(self.size_partition, self.size_partition)
    return U,V
      
#Finite Difference 2D:

#Laplacian
@jit(nopython=True)
def laplacian(Z, dx):
    Z_lap = np.zeros_like(Z)
    for i in range(1, Z.shape[0] - 1):
        for j in range(1, Z.shape[1] - 1):
            Z_lap[i, j] = (Z[i-1, j] + Z[i+1, j] + Z[i, j-1] + Z[i, j+1] - 4 * Z[i, j]) / dx**2
    return Z_lap

#main
@jit(nopython=True)
def main_null_flux(n, c, nu, h, a, b, du, dv, dt, U, V):
    for i in range(n):
        # laplaciano de U y V
        delta_U = laplacian(U, dx)
        delta_V = laplacian(V, dx)
        Uc = U[1:-1, 1:-1]
        Vc = V[1:-1, 1:-1]
        # Mesh change
        U[1:-1, 1:-1] = Uc + dt * (du * delta_U[1:-1, 1:-1] + nu*(Uc + a*Vc - c*Uc*Vc - Uc*Vc**2))
        V[1:-1, 1:-1] = Vc + dt * (dv * delta_V[1:-1, 1:-1] + nu*(h*Uc + b*Vc + c*Uc*Vc + Uc*Vc**2))

        # Null Flux
        for Z in (U, V):
            Z[0, :] = Z[1, :] 
            Z[-1, :] = Z[-2, :]
            Z[:, 0] = Z[:, 1]
            Z[:, -1] = Z[:, -2]
            
        if i == n - 1:
            return U

#Map
def generate_map(matrix,L,P):
    vmin = np.min(matrix)
    vmax = np.max(matrix)
    fig, ax = plt.subplots()
    ax.imshow(matrix[1:P+2,1:P+2], cmap = 'viridis',interpolation='spline36',extent=[0, L, 0, L],vmin=vmin, vmax=vmax)
    ax.set_axis_off()

schnakenberg = parameters(-0.37,-1.4,4,1,20,100,0.2,3) #c,nu,h,a,b,d1,d2,size,T,L

a,b,gamma,du,dv,size_partition,T,L,dx,dt,n= schnakenberg.values()

U,V = schnakenberg.initial_conditions()

result = main_null_flux(n, gamma, a, b, du, dv, dt, U, V)